In [ ]:
#include <bits/stdc++.h>
using namespace std;
typedef long long ll;
typedef long double ld;
typedef unsigned long long ull;

# Hello world 测试

In [ ]:
cout << "Hello world" << endl;

Hello world


# log10
`log10(x + 1)`可以计算`x`的位数

In [ ]:
int n = 27;
cout << ceil(log10(n + 1)) << endl;

2


# Repeat
`Repeat`可以重复字符串

In [ ]:
string Repeat(string Data, int Times)
{
    string Output = "";
    for (int i = 0; i < Times; i++)
        Output += Data;
    return Output;
}

In [ ]:
cout << Repeat("Hello", 5) << endl;

HelloHelloHelloHelloHello


##  树

In [26]:
#include <bits/stdc++.h>
using namespace std;
typedef long long ll;
typedef long double ld;
typedef unsigned long long ull;
const int MAXN = 100;
const int MAXLOG = 20;

### 树的存储

- 双亲表示法

In [27]:
int fa[MAXN], hd[MAXN], to[MAXN], nxt[MAXN], tot, id, s[MAXN], e[MAXN], dfs_seq[MAXN], id2, euler_seq[MAXN], dep[MAXN], anc[MAXN][MAXLOG];

In [28]:
void link(int father, int son) {
    fa[son] = father;
}

- 图存储法: 链式前向星

In [29]:
void link(int father, int son) {
    fa[son] = father;
    ++tot;
    to[tot] = son;
    nxt[tot] = hd[father];
    hd[father] = tot;
}

#### 树的深度优先遍历

In [30]:
void dfs(int x, int f) {
    fa[x] = f;
    dep[x] = dep[f] + 1;
    s[x] = ++id;
    dfs_seq[id] = x;
    euler_seq[++id2] = x;
    anc[x][0] = f;
    for (int i = 1; i <= MAXLOG; ++i) 
        anc[x][i] = anc[anc[x][i - 1]][i - 1];
    for (int i = hd[x]; i; i = nxt[i]) 
        if (to[i] != f) {
            dfs(to[i], x);
            euler_seq[++id2] = x;
        }
    e[x] = id;
}

$e_x-s_x=sz_x$

#### 最近公共祖先 `LCA`

##### 朴素算法 $O(n)$

In [31]:
int lca(int x, int y) {
    while (x != y) {
        if (dep[x] >= dep[y]) x = fa[x];
        else y = fa[y];
    }
    return x;
}

##### 倍增算法 $O(n\log n)$

In [25]:
int lca(int x, int y) {
    if (dep[x] < dep[y]) swap(x, y);
    for (int i = MAXLOG - 1; i >= 0; i--)
        if (dep[anc[x][i]] >= dep[y]) 
            x = anc[x][i];
    if (x == y) return x;
    for (int i = MAXLOG - 1; i >= 0; i--)
        if (anc[x][i] != anc[y][i])
            x = anc[x][i], y = anc[y][i];
    return anc[x][0];
}

### 二叉平衡(搜索)树

### Treap

- `v` 值
- `w` 权重——随机数
- `t` 相同的值的数量
- `s` 子树的数字总数
- `lc` 左子树下标
- `rc` 右子树下标

In [2]:
const int MAXN = 5000;
int root = 0, tot = 0;
int v[MAXN], w[MAXN];
int t[MAXN], s[MAXN];
int lc[MAXN], rc[MAXN];
int c[MAXN];

#### 插入节点

In [3]:
int new_node(int _v)
{
    tot++;
    v[tot] = _v;
    w[tot] = rand();
    c[tot] = s[tot] = 1;
    lc[tot] = rc[tot] = 0;
    return tot;
}

不同处：第5行w赋值为随机数

#### 更新

In [4]:
void update(int k)
{
    s[k] = s[lc[k]] + s[rc[k]] + c[k];
}

#### 旋转

##### 左旋

In [5]:
void lturn(int &k)
{
    int t = rc[k];
    rc[k] = lc[t];
    lc[t] = k;
    s[t] = s[k];
    update(k);
    k = t;
}

##### 右旋

In [6]:
void rturn(int &k)
{
    int t = lc[k];
    lc[k] = rc[t];
    rc[t] = k;
    s[t] = s[k];
    update(k);
    k = t;
}

##### 插入

In [7]:
void insert(int &k, int x)
{
    if (k == 0)
    {
        k = new_node(x);
        return;
    }
    s[k]++;
    if (v[k] == x)
        c[k]++;
    else if (v[k] < x)
    {
        insert(rc[k], x);
        if (w[rc[k]] < w[k])
            lturn(k);
    }
    else
    {
        insert(lc[k], x);
        if (w[lc[k]] < w[k])
            rturn(k);
    }
}

#### 删除

In [8]:
void del(int &k, int x)
{
    if (k == 0)
        return;
    if (v[k] == x)
    {
        if (c[k] > 1)
        {
            c[k]--;
            s[k]--;
        }
        else if (!lc[k] || !rc[k])
            k = lc[k] + rc[k];
        else
        {
            if (w[lc[k]] < w[rc[k]])
                rturn(k);
            else
                lturn(k);
            del(k, x);
        }
    }
    else
    {
        s[k]--;
        if (v[k] < x)
            del(rc[k], x);
        else
            del(lc[k], x);
    }
}

#### 查询第k大

In [9]:
int query_kth(int k, int x)
{
    if (k == 0)
        return 0;
    if (x <= s[lc[k]])
        return query_kth(lc[k], x);
    else if (x > s[lc[k]] + c[k])
        return query_kth(rc[k], x - s[lc[k]] - c[k]);
    else
        return v[k];
}

#### 查询前驱

In [11]:
int ans;

In [12]:
void query_pre(int k, int x)
{
    if (k == 0)
        return;
    if (v[k] < x)
    {
        ans = k;
        query_pre(rc[k], x);
    }
    else
        query_pre(lc[k], x);
}

#### 查询后继

In [14]:
void query_suc(int k, int x)
{
    if (k == 0)
        return;
    if (v[k] > x)
    {
        ans = k;
        query_suc(lc[k], x);
    }
    else
        query_suc(rc[k], x);
}

# C++14新特性

In [4]:
#include <bits/stdc++.h>
using namespace std;

In [8]:
pair<int, int> a = {1, 2};

In [1]:
auto [x, y] = a;
cout << x << " " << y << endl;

input_line_7:2:7: warning: decomposition declarations are a C++17 extension [-Wc++17-extensions]
 auto [x, y] = a;
      ^~~~~~
input_line_7:2:16: error: use of undeclared identifier 'a'
 auto [x, y] = a;
               ^
input_line_7:3:1: error: use of undeclared identifier 'cout'
cout << x << " " << y << endl;
^
input_line_7:3:26: error: use of undeclared identifier 'endl'; did you mean 'std::endl'?
cout << x << " " << y << endl;
                         ^~~~
                         std::endl
/home/langningc2009/miniconda3/envs/cling/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/9.4.0/ostream:599:5: note: 'std::endl' declared here
    endl(basic_ostream<_CharT, _Traits>& __os)
    ^


Interpreter Error: 

In [18]:
bool cmp(int x, int y)
{
    return x > y;
}
bool cmp2(int x, int y)
{
    return x < y;
}

In [22]:
int a[5] = {3, 5, 2, 4, 1};
sort(a, a + 5, cmp);
sort(a, a + 5, [](int x, int y) { return x > y; });
for (auto i : a) cout << i << " ";
cout << endl;

5 4 3 2 1 


In [23]:
// C++17 新特性
// 1. 常量表达式
constexpr int Temp1 = 1;
const int Temp2 = 1;
// 2. 结构化绑定
auto [x, y] = a;
// 3. if constexpr
if constexpr (sizeof(int) == 4) {
    // ...
}
// 4. 折叠表达式
template <typename... T>
auto sum(T... args) {
    return (args + ...);
}
// 5. lambda 表达式的 init-capture
int x = 1;
auto f = [y = x + 1]() { return y * y; };
// 6. constexpr if
template <typename T>
void f(T t) {
    if constexpr (is_integral_v<T>) {
        // ...
    } else {
        // ...
    }
}
// 7. 变量模板
template <typename T>
constexpr T pi = T(3.1415926535897932385L);
// 8. 类模板的模板参数推导
template <typename T>
struct A {
    A(T t) {}
};
A a(1); // C++17 之前需要 A<int> a(1);


input_line_30:7:6: warning: decomposition declarations are a C++17 extension [-Wc++17-extensions]
auto [x, y] = a;
     ^~~~~~
input_line_30:7:6: error: type 'int [5]' decomposes into 5 elements, but only 2 names were provided


Interpreter Error: 

In [25]:
pair<int, int> a = {1, 2};
// 17
pair a = {1, 2};

input_line_32:4:1: error: use of class template 'pair' requires template arguments
pair a = {1, 2};
^
/home/langningc2009/miniconda3/envs/cling/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/9.4.0/bits/stl_pair.h:193:56: note: template is declared here
    template<typename _T1, typename _T2> friend struct pair;
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~               ^


Interpreter Error: 

#  面积并

In [6]:
#define lson (x << 1)
#define rson (x << 1 | 1)
const int MAXN = 100005;
int sum[MAXN << 2], len[MAXN << 1];

In [8]:
int X[MAXN], Y[MAXN];

In [9]:
void pushup(int x, int l, int r)
{
    if (sum[x])
        len[x] = X[r + 1] - X[l];
    else if (l == r)
        len[x] = 0;
    else
        len[x] = len[x << 1] + len[x << 1 | 1];
}

In [10]:
void update(int x, int l, int r, int L, int R, int c)
{
    if (X[r + 1] <= L || R <= X[l])
        return;
    if (L <= X[l] && X[r + 1] <= R)
    {
        sum[x] += c;
        pushup(x, l, r);
        return;
    }
    int mid = (l + r) >> 1;
    update(lson, l, mid, L, R, c);
    update(rson, mid + 1, r, L, R, c);
    pushup(x, l, r);
}

In [15]:
int n;
struct
{
    double x, y1, y2;
    int c;
} line[MAXN];

In [16]:
scanf("%d", &n);
for (int i = 1, x1, y1, x2, y2; i <= n; i++)
{
    scanf("%d%d%d%d", &x1, &y1, &x2, &y2);
    X[2 * i - 1] = x1, X[2 * i] = x2;
    line[2 * i - 1] = {x1, y1, y2, 1};
    line[2 * i] = {x2, y1, y2, -1};
}
n <<= 1;
sort(line + 1, line + n + 1);
sort(X + 1, X + n + 1);
tot = unique(X + 1, X + n + 1) - X - 1;
for (int i = 1; i <= n; i++)
{
    update(1, 1, tot - 1, line[i].l, line[i].r, line[i].mark);
    ans += 1ll * len[i] * (line[i + 1].h - line[i].h);
}
printf("%lld\n", ans);

input_line_22:7:24: error: non-constant-expression cannot be narrowed from type 'int' to 'double' in initializer list [-Wc++11-narrowing]
    line[2 * i - 1] = {x1, y1, y2, 1};
                       ^~
input_line_22:7:24: note: insert an explicit cast to silence this issue
    line[2 * i - 1] = {x1, y1, y2, 1};
                       ^~
                       static_cast<double>( )
input_line_22:7:28: error: non-constant-expression cannot be narrowed from type 'int' to 'double' in initializer list [-Wc++11-narrowing]
    line[2 * i - 1] = {x1, y1, y2, 1};
                           ^~
input_line_22:7:28: note: insert an explicit cast to silence this issue
    line[2 * i - 1] = {x1, y1, y2, 1};
                           ^~
                           static_cast<double>( )
input_line_22:7:32: error: non-constant-expression cannot be narrowed from type 'int' to 'double' in initializer list [-Wc++11-narrowing]
    line[2 * i - 1] = {x1, y1, y2, 1};
                               ^~
input

Interpreter Error: 